## Training

In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

In [35]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
count = 0
videoFile = "/content/drive/MyDrive/Shou_Trumana_Kinosimka.RU (online-video-cutter.com).mp4"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [ ]:
i = 0 # i did this to do the mapping file in order to teach the neural net
while i < 301:
  i=str(i)
  img = plt.imread('frame'+i+'.jpg')   # reading image using its name
  print('frame',i,':')
  plt.imshow(img)
  plt.show()
  i=int(i)
  i+=1

In [5]:
data = pd.read_csv('mapping.csv')     # reading the csv file
data.head()      # printing first five rows of the file

,Image_ID,Class
0,frame0.jpg,0
1,frame1.jpg,0
2,frame2.jpg,0
3,frame3.jpg,0
4,frame4.jpg,1


In [6]:
X = [ ]     # creating an empty array
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array

In [7]:
y = data.Class
dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes

In [8]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

In [9]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X) 

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

In [11]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [12]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer

58900480/58889256 [==============================] - 1s 0us/step


In [13]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

((208, 7, 7, 512), (90, 7, 7, 512))

In [14]:
X_train = X_train.reshape(208, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(90, 7*7*512)

In [15]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

In [16]:
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

In [17]:
# ii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# iii. Training the model
model.fit(train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
7/7 [==============================] - 10s 230ms/step - loss: 0.6699 - accuracy: 0.6127 - val_loss: 0.3571 - val_accuracy: 0.8667
Epoch 2/10
7/7 [==============================] - 1s 153ms/step - loss: 0.1609 - accuracy: 0.9596 - val_loss: 0.2710 - val_accuracy: 0.8444
Epoch 3/10
7/7 [==============================] - 1s 158ms/step - loss: 0.0640 - accuracy: 0.9943 - val_loss: 0.2591 - val_accuracy: 0.8556
Epoch 4/10
7/7 [==============================] - 1s 155ms/step - loss: 0.0416 - accuracy: 1.0000 - val_loss: 0.2635 - val_accuracy: 0.8667
Epoch 5/10
7/7 [==============================] - 1s 154ms/step - loss: 0.0245 - accuracy: 1.0000 - val_loss: 0.2338 - val_accuracy: 0.8667
Epoch 6/10
7/7 [==============================] - 1s 153ms/step - loss: 0.0135 - accuracy: 1.0000 - val_loss: 0.2479 - val_accuracy: 0.8778
Epoch 7/10
7/7 [==============================] - 1s 153ms/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 0.2761 - val_accuracy: 0.8778
Epoch 8/10
7/7 [===